In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets.mnist import load_data

In [2]:
class MyDNN(tf.keras.Model):
    def __init__(self):
        super(MyDNN, self).__init__()
        self.W1 = tf.Variable(tf.random.normal([784, 500],stddev=0.03), trainable=True, name='weight_1')
        self.B1 = tf.Variable(tf.zeros([500]), trainable=True, name='bias_1')
        self.W2 = tf.Variable(tf.random.normal([500, 10],stddev=0.03), trainable=True, name='weight_2')
        self.B2 = tf.Variable(tf.zeros([10]), trainable=True, name='bias_2')
    
    @tf.function   
    def call(self, inputs):
        layer1 = tf.nn.relu(tf.matmul(inputs, self.W1) + self.B1)
        return tf.matmul(layer1, self.W2) + self.B2

In [3]:
model = MyDNN()

In [4]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)

In [5]:
@tf.function
def train_mnist(train_X,train_Y):
    train_X = tf.reshape(train_X, [-1,784])
    train_Y = tf.cast(train_Y, tf.int32)
#     train_Y = tf.one_hot(train_Y, 10)
    with tf.GradientTape() as tape:
        predict_labels = model(train_X) 
        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=predict_labels, labels=train_Y))
    layer_variables = model.trainable_variables
    grads = tape.gradient(loss, layer_variables)
    optimizer.apply_gradients(zip(grads, layer_variables))
    if optimizer.iterations % 100 == 0:
#         tf.print("grads: ", grads)
#         tf.print("w1:", model.W1)
#         tf.print("w2: ",model.W2)
#         tf.print("layer_variables: ",layer_variables)
        
        tf.print("After ", optimizer.iterations,end="")
        tf.print(" steps , loss is ",loss)

In [6]:
mnist = load_data("mnist.npz")
train,test = mnist
train_ds = tf.data.Dataset.from_tensor_slices(train).batch(100).repeat()

In [7]:
writer = tf.summary.create_file_writer("./logs")
tf.summary.trace_on(graph=True, profiler=True)

Instructions for updating:
use `tf.profiler.experimental.start` instead.


In [8]:
for train_X, train_Y in train_ds:
    if optimizer.iterations > 6000:
        break
    train_mnist(tf.cast(train_X,tf.float32), train_Y)

After  100 steps , loss is  0.431293041
After  200 steps , loss is  0.448698133
After  300 steps , loss is  0.298939824
After  400 steps , loss is  0.519782841
After  500 steps , loss is  0.183281884
After  600 steps , loss is  0.240476325
After  700 steps , loss is  0.170609549
After  800 steps , loss is  0.142453313
After  900 steps , loss is  0.146287337
After  1000 steps , loss is  0.222209126
After  1100 steps , loss is  0.129638046
After  1200 steps , loss is  0.202660814
After  1300 steps , loss is  0.0613217503
After  1400 steps , loss is  0.112634972
After  1500 steps , loss is  0.0857687891
After  1600 steps , loss is  0.0970524
After  1700 steps , loss is  0.136885345
After  1800 steps , loss is  0.201414734
After  1900 steps , loss is  0.0429092348
After  2000 steps , loss is  0.07485535
After  2100 steps , loss is  0.0903446302
After  2200 steps , loss is  0.0851831064
After  2300 steps , loss is  0.0888379738
After  2400 steps , loss is  0.169405043
After  2500 steps , lo

In [9]:
with writer.as_default():
    tf.summary.trace_export(
        name="my_func_trace",
        step=0,
        profiler_outdir="./logs")

Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.


In [121]:
test_X, test_Y = test
test_X = tf.cast(tf.reshape(test_X,[-1, 784]), tf.float32)
test_predict = model(test_X)
correct_prediction = tf.equal(test_Y, tf.argmax(test_predict, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("accuracy: ",accuracy)

accuracy:  tf.Tensor(0.9587, shape=(), dtype=float32)
